## Importing Packages

In [47]:
from pulp import *
import pandas as pd
import numpy as np
import calendar

## Declaring Variables

In [37]:
time = np.arange(1, 13)   # time corresponding to each month

workforce_bfr = 64000       # initial number of people in workforce
inventory_bfr = 1000000     # initial amount of inventory

work_days = np.linspace(20, 20, 12)    # working days array
demand = [144000000, 134400000, 123600000, 144000000, 108000000, 72000000, 72000000, 72000000, 120000000, 108000000, 144000000, 120000000]  # demand array


num_hired = {i: LpVariable(name=f"num_hired{i}", lowBound=0) for i in range(1, 13)}             # number of people hired
num_fired = {i: LpVariable(name=f"num_fired{i}", lowBound=0) for i in range(1, 13)}             # number of people fired
num_inventory = {i: LpVariable(name=f"num_inventory{i}", lowBound=0) for i in range(0, 13)}     # inventory level
num_production = {i: LpVariable(name=f"num_production{i}", lowBound=0) for i in range(1, 13)}   # production level
num_workforce = {i: LpVariable(name=f"num_workforce{i}", lowBound=0) for i in range(0, 13)}     # number of people in the workforce
num_subcontract = {i: LpVariable(name=f"num_subcontract{i}", lowBound=0) for i in range(1, 13)} # number of units subcontracted

## Initializing the Model

In [38]:
# Model
model = LpProblem(name="cost-problem", sense=LpMinimize)

## Adding Constraints to the Model

In [41]:
# Non-negative constraints
for t in range(1, 13):
    #print(t)
    model += (num_hired[t] >= 0,       "")
    model += (num_fired[t] >= 0,       "")
    model += (num_inventory[t] >= 0,   "")
    model += (num_production[t] >= 0,  "")
    model += (num_workforce[t] >= 0,   "")
    model += (num_subcontract[t] >= 0, "")

# Terminate constraint
model += num_inventory[12] >= 24000000

# Intial workforce and inventory constraints
model += (num_workforce[0] == 64000)
model += (num_inventory[0] == 1000000)

# Relationship between production and workforce
for t in range(1, 13):
    #print(t)
    model += (num_production[t] == (8 * 12) * work_days[t-1] * num_workforce[t],  "")

# Conservation of inventory level
for t in range(1, 13):
    #print(t)
    model += (num_inventory[t] == num_inventory[t-1] + num_production[t] + num_subcontract[t] - demand[t-1],  "")

# Conservation of workforce level
for t in range(1, 13):
    #print(t)
    model += (num_workforce[t] == num_workforce[t-1] + num_hired[t] - num_fired[t],  "")


## Setting the Objective Function

In [42]:
# Set objective
model += lpSum([(320 * num_hired[t]) + (600 * num_fired[t]) + (0.5 * num_inventory[t]) + (4 * num_production[t]) + (4.2 * num_subcontract[t]) for t in range(1, 13)])

C:\Users\conor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pulp\pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


## Solving the Problem

In [54]:
# Solve the optimization problem
status = model.solve()

## Optimal Number of People Hired for Each Month

In [55]:
num = 1
for var in num_hired.values():
    print(f"Number of People Hired in {calendar.month_name[num]}: {var.value()}")
    num += 1

Number of People Hired in January: 375.0
Number of People Hired in February: 0.0
Number of People Hired in March: 0.0
Number of People Hired in April: 0.0
Number of People Hired in May: 0.0
Number of People Hired in June: 0.0
Number of People Hired in July: 0.0
Number of People Hired in August: 0.0
Number of People Hired in September: 18750.0
Number of People Hired in October: 0.0
Number of People Hired in November: 18750.0
Number of People Hired in December: 0.0


## Optimal Number of People Fired for Each Month

In [52]:
num = 1
for var in num_fired.values():
    print(f"Number of People Fired in {calendar.month_name[num]}: {var.value()}")
    num += 1

Number of People Fired in January: 0.0
Number of People Fired in February: 0.0
Number of People Fired in March: 0.0
Number of People Fired in April: 0.0
Number of People Fired in May: 8125.0
Number of People Fired in June: 18750.0
Number of People Fired in July: 0.0
Number of People Fired in August: 0.0
Number of People Fired in September: 0.0
Number of People Fired in October: 0.0
Number of People Fired in November: 0.0
Number of People Fired in December: 0.0


## Optimal Number of Units Subcontracted for Each Month

In [53]:
num = 1
for var in num_subcontract.values():
    print(f"Number of Units Subcontracted in {calendar.month_name[num]}: {var.value()}")
    num += 1

Number of Units Subcontracted in January: 19400000.0
Number of Units Subcontracted in February: 10800000.0
Number of Units Subcontracted in March: 0.0
Number of Units Subcontracted in April: 20400000.0
Number of Units Subcontracted in May: 0.0
Number of Units Subcontracted in June: 0.0
Number of Units Subcontracted in July: 0.0
Number of Units Subcontracted in August: 0.0
Number of Units Subcontracted in September: 12000000.0
Number of Units Subcontracted in October: 0.0
Number of Units Subcontracted in November: 0.0
Number of Units Subcontracted in December: 0.0


## Minimized Total Cost

In [58]:
# Get the results
print(f"Minimized Total Cost: ${model.objective.value():,.2f}")

Minimized Total Cost: $5,592,765,000.00
